# Assignment: Ebay trending products

In this assignment you gonna scrape Ebay trending products page using Requests and LXML.

## Assignemnt instructions

**Target web page**: https://www.ebay.com/trending -> atualmente é https://www.ebay.com/globaldeals/trending/all

In this assignment you will **scrape all the trending products from Ebay**, each trending product looks like this:

![war of the spark](https://img-c.udemycdn.com/redactor/raw/2019-05-02_19-06-59-edd90983178656d3901c05b668e2afe1.PNG)

We have the **title**(which contains the **URL** too), the **product description**, **amount of searches** and some **related items** to the product itself.

From each trending product I want you to scrape the following:

1. Title
2. URL
3. Description(info)
4. Amount of searches

Finally make sure to store all the scraped trending products in a JSON file.

A sample of a JSON file with one trending products(yours MUST contain ALL the products)

```json
{
  "name": "War of the Spark",
  "url": "https://www.ebay.com/sch/i.html?rt=nc&_nkw=War+of+the+Spark&_trksid=p2245348.m3768.l6570&_trkparms=topicId%3D7389&_trksid=p2245348.m3768.l6569&_trkparms=topicId%3D7389",
  "info": "Wizards of the Coast is calling War of the Spark its most ambitious Magic: The Gathering set ever. The extra special Mythic Edition includes eight Masterpiece cards with foil designs and full-bleed artwork.",
  "searches": "10599"
}
```

If you did face some issues or you couldn't understand something please ask me in the Q&A area and make sure to use [Assignment] tag.

## Meu código:

In [49]:
import requests
import re

from lxml import html

import json
import csv

from datetime import date

Definindo as funcoes de escrever arquivos

In [46]:
todayString = '[' + date.today().strftime("%Y-%m-%d--%H-%M") + ']'


def writeToJson(products):
    with open(f"{todayString} - products.json", 'w', newline='', encoding='UTF-8') as f:
        f.write(json.dumps(products, ensure_ascii=False))


def writeToCSV(products, headers):
    with open(f"{todayString} - products.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, headers)
        writer.writeheader()
        writer.writerows(products)


In [ ]:
resp = requests.get(url='https://www.ebay.com/globaldeals/trending/all')
print(resp.text)

In [96]:
tree = html.fromstring(html=resp.text)

all_products = []

trending_items = tree.xpath(
    "//div[contains(@itemscope, 'itemscope')]/div[@class='dne-itemtile-detail']")

for trending_item in trending_items:
    item = {
        'name': ''.join(map(str.strip, trending_item.xpath(".//h3//span[@itemprop='name']/text()"))),
        'url': trending_item.xpath(".//a[@itemprop='url']/@href")[0],
        'price': trending_item.xpath(".//div[contains(@class, 'price')]//span[@itemprop='price']/text()")[0],
        'priceNumber': ''
    }
    formatedPrice = ''.join(re.findall(r"\S", item['price']))
    item['priceNumber'] = ''.join(re.compile(r'(\d+[(\.|,)]\d+)').findall(formatedPrice))
    all_products.append(item)

# print(all_products)

writeToJson(all_products)
writeToCSV(products=all_products, headers=[
           'name', 'url', 'price', 'priceNumber'])
